In [1]:
import os
import pandas as pd
import numpy as np
from utils import to_21d_vec, get_beat_vector
import re
import sys
sys.path.insert(1, './harmalysis')
from inputRN2Chord import inputRN

In [6]:
script_dir = os.getcwd()
score_rel_path = "../datasets/haydn_op20_harm/haydn_org_score_for_vec/op20n6-04_upm3_trans_vector_pre.krn"
scorepath = os.path.join(script_dir, score_rel_path)


In [7]:
df = pd.read_csv(scorepath, sep="\t", header=None)
pd.set_option('display.max_rows', df.shape[0]+1)
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "beat", "measure", "key", "meter" ]

In [8]:
#process dataframe, delete bars, score representations, and "rest slice"
df = df[~df['beat'].astype(str).str.startswith(('=','.','*'))]
df

,harm,voice4,voice3,voice2,voice1,beat,measure,key,meter
2,I,1r,1r,2.r,8gg#,1.5,.,C#,M4/4
3,I,1r,1r,2.r,8gg#,2,.,C#,M4/4
4,I,1r,1r,2.r,8gg#,2.5,.,C#,M4/4
5,I,1r,1r,2.r,8ee#,3,.,C#,M4/4
6,I,1r,1r,2.r,16dd#,3.5,.,C#,M4/4
7,I,1r,1r,2.r,16ee#,3.75,.,C#,M4/4
8,(I),1r,1r,[4cc#,8cc#,4,.,C#,M4/4
9,(I),1r,1r,[4cc#,8ee#,4.5,.,C#,M4/4
11,(I),1r,1r,4cc#],8dd#,1,=2,C#,M4/4
12,(I),1r,1r,4cc#],4ddd#,1.5,=2,C#,M4/4


In [9]:
prev_note_list = []
for index, row in df.iterrows():
    # for this onset slice 
    this_onset_vec = []
    
    ######################################### Process notes and onsets ########################################
    voices = ["voice4","voice3","voice2","voice1"]
    cur_note_tmp = []
    for v in voices:
        this_note = ''.join(row[[v]].values)
        if ' ' in this_note: #for a part has multiple voices
            this_note_list = this_note.split(" ")
            cur_note_tmp.extend(this_note_list)
        else:
            cur_note_tmp.append(this_note)
    
    cur_note_list = []
    for n in cur_note_tmp:
        note_name = re.sub('[^a-gA-G#-]+', '', n)
        if note_name:
            cur_note_list.append(note_name)
            
    cur_onset_list = list(set(cur_note_list) - set(prev_note_list))
    prev_note_list = cur_note_list
    
    
    note_vec = to_21d_vec(cur_note_list)
    onset_vec = to_21d_vec(cur_onset_list)
    this_onset_vec.extend(note_vec)
    this_onset_vec.extend(onset_vec)
    
    ######################################### Process beat position ########################################
    beat_pos = ''.join(row[["beat"]].values)
    meter = ''.join(row[["meter"]].values)
    meter = meter.replace("M", "")
    beat_vec = get_beat_vector(beat_pos, meter)
    this_onset_vec.extend(beat_vec)
    
    ######################################### Process chord label ########################################
    harm = ''.join(row[["harm"]].values)
    harm = re.sub('[()]', '', harm) # process string
    key = ''.join(row[["key"]].values)
    key_harm = key + ":" + harm
    #print(key_harm)
    try:
        chord_label = inputRN(key_harm)["Chord label"]
    except:
        continue # pass this onset slice if RN is not recognizable
    this_onset_vec.append(str(chord_label))
    #print(this_onset_vec)
  
    #########################################################################################################


gg# G#
gg# G#
gg# G#
gg# G#
ee# E#
ee# E#
dd# D#
dd# D#
ee# E#
ee# E#
cc# C#
cc# C#
cc# C#
cc# C#
ee# E#
ee# E#
cc# C#
dd# D#
dd# D#
cc# C#
ddd# D#
ddd# D#
b# B#
ddd# D#
b# B#
b# B#
bb# B#
bb# B#
b# B#
gg# G#
gg# G#
b# B#
ccc# C#
ccc# C#
b# B#
cc# C#
cc# C#
a# A#
a# A#
a# A#
cc# C#
cc# C#
a# A#
b# B#
b# B#
a# A#
bb# B#
bb# B#
g# G#
bb# B#
g# G#
g# G#
gg# G#
gg# G#
g# G#
ee# E#
ee# E#
g# G#
aa# A#
aa# A#
g# G#
a# A#
a# A#
f# F#
f# F#
f# F#
a# A#
a# A#
f# F#
g# G#
g# G#
f# F#
gg# G#
gg# G#
e# E#
gg# G#
e# E#
e# E#
ee# E#
ee# E#
e# E#
cc# C#
cc# C#
d# D#
ff# F#
d# D#
ff# F#
c# C#
gg# G#
c# C#
gg# G#
c# C#
aa# A#
aa# A#
B# B#
gg# G#
B# B#
gg# G#
B# B#
ff# F#
ff# F#
c# C#
ee# E#
ee# E#
c# C#
cc# C#
ee# E#
cc# C#
cc# C#
ff## G
ff## G
cc# C#
ff## G
cc# C#
ee# E#
ee# E#
cc# C#
ff## G
ff## G
b# B#
gg# G#
b# B#
gg# G#
a# A#
gg# G#
a# A#
b# B#
gg# G#
b# B#
g# G#
g# G#
g# G#
g# G#
b# B#
b# B#
g# G#
a# A#
a# A#
g# G#
aa# A#
aa# A#
f## G
aa# A#
f## G
f## G
ff## G
aa# A#
ff## G
f## G
dd# D#
aa# A#
dd

c# C#
f# F#
dd# D#
f# F#
dd# D#
c# C#
g# G#
ee# E#
ee# E#
g# G#
A# A#
a# A#
ff# F#
a# A#
A# A#
ff# F#
G# G#
cc# C#
ff# F#
cc# C#
G# G#
A# A#
cc# C#
ff# F#
A# A#
F# F#
ff# F#
F# F#
A# A#
ff# F#
cc# C#
cc# C#
A# A#
G# G#
ff# F#
b B
G# G#
b B
g# G#
ff# F#
dd# D#
bb B
bb B
g# G#
dd# D#
g# G#
ee# E#
ee# E#
bb B
ee# E#
e# E#
ee# E#
gg# G#
gg# G#
e# E#
gg# G#
c# C#
ee# E#
gg# G#
ee# E#
c# C#
F# F#
ee# E#
aa# A#
F# F#
aa# A#
f# F#
ee# E#
cc# C#
a# A#
a# A#
cc# C#
f# F#
dd# D#
dd# D#
dd# D#
F# F#
dd# D#
ff# F#
a# A#
a# A#
F# F#
ff# F#
E# E#
dd# D#
g# G#
E# E#
g# G#
e# E#
dd# D#
b# B#
gg# G#
e# E#
b# B#
gg# G#
e# E#
cc# C#
cc# C#
gg# G#
cc# C#
c# C#
cc# C#
ee# E#
ee# E#
ee# E#
c# C#
A# A#
cc# C#
ee# E#
cc# C#
A# A#
d# D#
cc# C#
ff# F#
d# D#
ff# F#
D# D#
cc# C#
ff# F#
f# F#
D# D#
f# F#
b# B#
ff# F#
b# B#
GG# G#
b# B#
ff# F#
dd# D#
GG# G#
dd# D#
C# C#
cc# C#
ee# E#
cc# C#
ee# E#
cc# C#
C# C#
C# C#
cc# C#
dd# D#
ccc# C#
dd# D#
ccc# C#
C# C#
cc# C#
ee# E#
ccc# C#
ee# E#
cc# C#
ccc# C#
ee# E#
ccc# C#

D# D#
ff## G
ff## G
cc# C#
D# D#
G# G#
ff## G
bb# B#
b# B#
bb# B#
b# B#
G# G#
GG# G#
dd# D#
b# B#
bb# B#
GG# G#
dd# D#
GG# G#
b# B#
b# B#
bb# B#
ee# E#
bb# B#
ee# E#
GG# G#
ee# E#
b# B#
gg# G#
GG# G#
b# B#
gg# G#
GG# G#
ee# E#
b# B#
ee# E#
FF## G
ee# E#
a# A#
aa# A#
FF## G
a# A#
aa# A#
F## G
e# E#
aa# A#
a# A#
F## G
e# E#
F## G
d# D#
aa# A#
d# D#
D# D#
dd# D#
ff## G
a# A#
a# A#
D# D#
ff## G
dd# D#
BB# B#
dd# D#
dd# D#
a# A#
BB# B#
E# E#
dd# D#
gg# G#
g# G#
E# E#
g# G#
gg# G#
EE# E#
b# B#
g# G#
gg# G#
EE# E#
b# B#
EE# E#
g# G#
g# G#
gg# G#
cc# C#
gg# G#
cc# C#
E# E#
cc# C#
g# G#
ee# E#
ee# E#
E# E#
g# G#
E# E#
cc# C#
g# G#
cc# C#
D# D#
cc# C#
f## G
ff## G
f## G
D# D#
ff## G
d# D#
dd# D#
ff## G
ff## G
d# D#
dd# D#
d# D#
cc# C#
ff## G
ff## G
cc# C#
d# D#
b# B#
ff## G
ff## G
b# B#
B# B#
b# B#
dd# D#
gg# G#
dd# D#
B# B#
gg# G#
G# G#
b# B#
b# B#
ff## G
ff## G
G# G#
c# C#
b# B#
ee# E#
ee# E#
ee# E#
c# C#
C# C#
cc# C#
e# E#
ee# E#
cc# C#
C# C#
e# E#
C# C#
b# B#
e# E#
ee# E#
b# B#
a# A#
ee# E#


F# F#
b B
ccc# C#
b B
F# F#
g# G#
ccc# C#
g# G#
F# F#
a# A#
ccc# C#
a# A#
f# F#
aa# A#
f# F#
aa# A#
f# F#
f# F#
cc# C#
gg# G#
cc# C#
gg# G#
f# F#
f# F#
cc# C#
aa# A#
aa# A#
d# D#
dd# D#
ff# F#
d# D#
ff# F#
dd# D#
c# C#
dd# D#
ee# E#
ee# E#
c# C#
d# D#
dd# D#
ff# F#
d# D#
ff# F#
B# B#
dd# D#
dd# D#
B# B#
A# A#
g# G#
cc# C#
cc# C#
A# A#
g# G#
B# B#
g# G#
dd# D#
B# B#
dd# D#
G# G#
gg# G#
b# B#
G# G#
b# B#
gg# G#
e# E#
g# G#
gg# G#
gg# G#
e# E#
g# G#
c# C#
ee# E#
ee# E#
ee# E#
c# C#
B# B#
ee# E#
gg# G#
dd# D#
gg# G#
B# B#
dd# D#
c# C#
ee# E#
gg# G#
ee# E#
c# C#
A# A#
ccc# C#
cc# C#
cc# C#
A# A#
ccc# C#
f# F#
cc# C#
aa# A#
f# F#
aa# A#
e# E#
cc# C#
gg# G#
e# E#
gg# G#
d# D#
ff# F#
ff# F#
d# D#
ff# F#
e# E#
ff# F#
gg# G#
e# E#
gg# G#
c# C#
a# A#
ff# F#
ee# E#
a# A#
ee# E#
c# C#
d# D#
a# A#
ff# F#
ff# F#
d# D#
B# B#
ff# F#
dd# D#
B# B#
dd# D#
c# C#
ff# F#
ee# E#
ee# E#
c# C#
A# A#
ff# F#
d# D#
cc# C#
cc# C#
A# A#
d# D#
B# B#
ff# F#
d# D#
dd# D#
B# B#
dd# D#
G# G#
b# B#
b# B#
b# B#
G# G#
A# A#

In [9]:
print(inputRN ("A--:I")["Chord label"])

Abb major
